# Saving Models

Using the previous data (voice.csv) as an example, these notes will show how to save models.

In [21]:
# Step 1: Import Dependencies

import numpy as np
import pandas as pd

In [22]:
# Step 2: Load the Data

voice = pd.read_csv("resources/voice.csv")
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [23]:
# Step 3: Assign X and y

X = voice.drop("label", axis=1)
y = voice["label"]
print(X.shape, y.shape)

(3168, 20) (3168,)


In [24]:
# Step 4: Train Test Split the data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

### Preprocess the data (scale and one hot encode the data)

In [25]:
# Step 5: Use StandardScaler to scale X

from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Step 6: Use Label Encoder to encode the y before one hot encoding

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [27]:
import tensorflow

In [28]:
# Step 7: Use one hot encoding to make the labels legible tot he computer

from keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Neural Network

In [29]:
# Step 8: Create the model using Sequential and Dense

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [30]:
# Step 9: Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
# Step 10: Fit (train) the preprocessed data to the model

model.fit(
    X_train_scaled,
    y_train_categorical, 
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
75/75 - 0s - loss: 0.2453 - accuracy: 0.9045
Epoch 2/100
75/75 - 0s - loss: 0.0873 - accuracy: 0.9752
Epoch 3/100
75/75 - 0s - loss: 0.0717 - accuracy: 0.9769
Epoch 4/100
75/75 - 0s - loss: 0.0661 - accuracy: 0.9790
Epoch 5/100
75/75 - 0s - loss: 0.0582 - accuracy: 0.9802
Epoch 6/100
75/75 - 0s - loss: 0.0540 - accuracy: 0.9802
Epoch 7/100
75/75 - 0s - loss: 0.0505 - accuracy: 0.9806
Epoch 8/100
75/75 - 0s - loss: 0.0465 - accuracy: 0.9815
Epoch 9/100
75/75 - 0s - loss: 0.0438 - accuracy: 0.9832
Epoch 10/100
75/75 - 0s - loss: 0.0385 - accuracy: 0.9848
Epoch 11/100
75/75 - 0s - loss: 0.0375 - accuracy: 0.9857
Epoch 12/100
75/75 - 0s - loss: 0.0382 - accuracy: 0.9848
Epoch 13/100
75/75 - 0s - loss: 0.0313 - accuracy: 0.9861
Epoch 14/100
75/75 - 0s - loss: 0.0286 - accuracy: 0.9907
Epoch 15/100
75/75 - 0s - loss: 0.0285 - accuracy: 0.9903
Epoch 16/100
75/75 - 0s - loss: 0.0251 - accuracy: 0.9899
Epoch 17/100
75/75 - 0s - loss: 0.0257 - accuracy: 0.9907
Epoch 18/100
75/75 - 0s

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 12,402
Trainable params: 12,402
Non-trainable params: 0
_________________________________________________________________


# Quantify the Model

In [33]:
# Step 11: Quantify the model using preprocessed test data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2
)

25/25 - 0s - loss: 0.0648 - accuracy: 0.9861


# Save the Model (make sure it is trained)

In [34]:
# Step 11a: Saving the model
model.save("models/voice_model_trained_28.h5")

# Load the model

In [35]:
# Step 11b: Loading the model
from keras.models import load_model
voice_model = load_model("models/voice_model_trained_28.h5")

In [36]:
# Step 11c: evaluating the loaded model
model_loss, model_accuracy = voice_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2
)

25/25 - 0s - loss: 0.0648 - accuracy: 0.9861


# Make Predictions

In [37]:
# Step 12: take the top 5 values in the test scaled data to make predictions

encoded_predictions = voice_model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

C:\Users\gianm\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [38]:
print(f"Predicted Classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted Classes: ['female' 'female' 'male' 'female' 'male']
Actual Labels: ['female', 'female', 'male', 'female', 'male']
